In [1]:
from pymilvus import MilvusClient

_client = MilvusClient("milvus_demo.db")

In [2]:
_client.has_collection("collection_01")

True

In [3]:
_client.drop_collection(collection_name="collection_01")

In [2]:
_client.create_collection(collection_name="collection_01",dimension=1024)

In [5]:
# import os
# os.environ["http_proxy"]="http://192.168.10.29:7890"
# os.environ["https_proxy"]="http://192.168.10.29:7890"

In [5]:
# from pymilvus.model.hybrid import BGEM3EmbeddingFunction

# _emb = BGEM3EmbeddingFunction(
#     model_name="BAAI/bge-m3",
#     device="cuda",
#     use_fp16=True
# )
from langchain_community.embeddings import OllamaEmbeddings
_emb = OllamaEmbeddings(
    base_url="http://192.168.10.11:60006",
    model="bge-m3"
)

In [7]:
docs_embeddings = _emb.embed_documents(["天安门在哪里","北京"])
len(docs_embeddings[0]),type(docs_embeddings),type(docs_embeddings[0])

(1024, list, list)

In [18]:
docs = [
"天安门在北京",
"武侯祠在成都",
"故宫在北京"
]

vectors = _emb.embed_documents(docs)

datas = [
    {"id": i, "vector": vectors[i], "text": docs[i], "subject": "history"} for i in range(len(vectors))
]

print("Data has", len(datas), "entities, each with fields: ", datas[0].keys())
print("Vector dim:", len(datas[0]["vector"]))
print(datas[1]["text"])

Data has 3 entities, each with fields:  dict_keys(['id', 'vector', 'text', 'subject'])
Vector dim: 1024
武侯祠在成都


In [19]:
res = _client.insert(collection_name="collection_01", data=datas)

print(res)

{'insert_count': 3, 'ids': [0, 1, 2]}


In [34]:
query_vectors = _emb.embed_documents(["天安门在哪里?","武侯祠在哪？"])

query_vectors

[[1.1583247184753418,
  0.3732200562953949,
  -1.0804625749588013,
  0.06485140323638916,
  -0.4999196529388428,
  0.1861029863357544,
  -0.9047397375106812,
  0.36883294582366943,
  0.051554009318351746,
  1.4852674007415771,
  0.4372360110282898,
  0.08458073437213898,
  -0.17608213424682617,
  0.13416942954063416,
  0.08807948976755142,
  0.030939389020204544,
  1.5474562644958496,
  -0.493384450674057,
  -0.11614224314689636,
  -0.7679118514060974,
  -0.3381252586841583,
  -0.31283313035964966,
  -0.993503212928772,
  -0.14509962499141693,
  0.8581520915031433,
  0.2826818823814392,
  -0.0760507807135582,
  0.3858712613582611,
  -1.2693570852279663,
  -0.18587039411067963,
  -1.430911660194397,
  -0.3511761426925659,
  -0.6497582197189331,
  -0.5686365365982056,
  0.8031050562858582,
  -0.7957371473312378,
  -0.18567557632923126,
  0.4179643988609314,
  -1.728182315826416,
  0.17427699267864227,
  0.41677331924438477,
  0.20957884192466736,
  0.3663681745529175,
  -0.87689906358718

In [36]:
res = _client.search(
    collection_name="collection_01",
    data=query_vectors,
    limit=2,
    # filter=""
    output_fields=["text", "subject"],
)

print(res)

data: ["[{'id': 0, 'distance': 0.8503330945968628, 'entity': {'text': '天安门在北京', 'subject': 'history'}}]", "[{'id': 1, 'distance': 0.9020527601242065, 'entity': {'text': '武侯祠在成都', 'subject': 'history'}}]"] 
